In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import shutil
# shutil.rmtree("/kaggle/working/test1")
# shutil.rmtree("/kaggle/working/log")


/kaggle/input/dogs-vs-cats/test1.zip
/kaggle/input/dogs-vs-cats/train.zip
/kaggle/input/dogs-vs-cats/sampleSubmission.csv
/kaggle/input/doooog/dog.jpg


In [2]:
pip install tflearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=c847dc3c213a323e174e23cfae4d3d8f5b7def0ab73692ffe1f7738ef5ef3f28
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import zipfile
import cv2
import os
import matplotlib.pyplot as plt
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, fully_connected, dropout
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression

In [ ]:
with zipfile.ZipFile("../input/dogs-vs-cats/test1.zip","r") as z:
    z.extractall(".")

with zipfile.ZipFile("../input/dogs-vs-cats/train.zip","r") as z:
    z.extractall(".")

In [ ]:
TRAIN_DIR = 'train'
TEST_DIR = 'test1'
IMG_SIZE=50
LR = 0.001
MODEL_NAME = 'dog-vs-cats-cnn'

In [ ]:
def create_label(img_name):
    label = img_name.split('.')[-3]
    if label=='cat':
        return np.array([1,0])
    elif label=='dog':
        return np.array([0,1])


In [ ]:
def create_train_data():
    training_data=[]
    for img in tqdm(os.listdir(TRAIN_DIR)):
        path = os.path.join(TRAIN_DIR, img)
        img_data = cv2.imread(path,0)
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img_data), create_label(img)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

In [ ]:
if os.path.exists('train_data.npy'):
    train= np.load('train_data.npy', allow_pickle=True)
else:
    train = create_train_data()
    
x_train = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_train = [i[1] for i in train]

x_train = x_train/255

In [ ]:
# MODEL CONSTRUCTION
tf.compat.v1.reset_default_graph()
conv_input = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

conv1 = conv_2d(conv_input, 32, 5, activation='relu')
pool1 = max_pool_2d(conv1,5)

conv2 = conv_2d(pool1, 64, 5, activation= 'relu')
pool2 = max_pool_2d(conv2, 5)

conv3 = conv_2d(pool1, 128, 5, activation= 'relu')
pool3 = max_pool_2d(conv2, 5)

conv4 = conv_2d(pool1, 64, 5, activation= 'relu')
pool4 = max_pool_2d(conv2, 5)

conv5 = conv_2d(pool1, 32, 5, activation= 'relu')
pool5 = max_pool_2d(conv2, 5)

fully_layer = fully_connected(pool5, 1024, activation='relu')
fully_layer = dropout(fully_layer, 0.5)
cnn_layers = fully_connected(fully_layer, 2, activation='softmax') 

cnn_layers = regression(cnn_layers, learning_rate=LR, name='targets')
model = tflearn.DNN(cnn_layers, tensorboard_dir='log', tensorboard_verbose=3)

In [ ]:
if (os.path.exists('model.tfl.meta')):
    model.load('./model.tfl')
else:
    model.fit({'input': x_train}, {'targets': y_train}, n_epoch=10, #input, target, n_epochs are mandatory
              snapshot_step=500, show_metric=True, run_id=MODEL_NAME)
    model.save('model.tf1')

In [ ]:
def create_test_data():
    testing_data=[]
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR, img)
        img_data = cv2.imread(path,0)
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        testing_data.append([np.array(img_data)])
    np.save('test_data.npy', testing_data)
    return testing_data

if os.path.exists('test_data.npy'):
    test = np.load('test_data.npy', allow_pickle=True)
else:
    test = create_test_data()


x_test = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
x_test = x_test/255

In [ ]:
predicted = model.predict(x_test)
df = pd.read_csv('/kaggle/input/dogs-vs-cats/sampleSubmission.csv')
my_submission = pd.DataFrame({'Id': df.id, 'SalePrice': predicted})
my_submission.to_csv('submission.csv', index=False)

In [ ]:
print(my_submission)